### Manipulation of PEAKS de novo results of _T. weiss_  LC-MS/MS data using Python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >80% ALC
    from Thermo Fusion tribrid runs at the UW Proteomics Resource center, January 2021
    combined from multiple injections

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    Files with peptides and PTMs for PTM+cellular compartment x-analysis
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
cd /home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_324-T5dig-all_DENOVO_78/

/home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_324-T5dig-all_DENOVO_78


In [3]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks324 = pd.read_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_324-T5dig-all_DENOVO_78/324-T5dig-all_DENOVO_78_DN.csv")

print("# redundant Peaks peptides >80% ALC in combined dataframe:", len(peaks324))

# These columns mess things up- get rid of them

del peaks324['Fraction']
del peaks324['Scan']
del peaks324['Source File']
del peaks324['Tag Length']
del peaks324['PTM']
del peaks324['tag (>=0%)']
del peaks324['mode']
del peaks324['local confidence (%)']

mean_len = peaks324['length'].mean()
print(mean_len)

# look at the dataframe
peaks324.head()

# redundant Peaks peptides >80% ALC in combined dataframe: 4201
8.10449892882647


,Peptide,ALC (%),length,m/z,z,RT,Area,Mass,ppm
0,KFDEETK,99,7,448.7220,2,17.90,1710000.0,895.4286,0.9
1,TPEEEER,99,7,445.1992,2,16.15,52200.0,888.3824,1.6
2,HLDVDDSGK,98,9,493.2327,2,25.39,180000.0,984.4512,-0.4
3,TPEEEER,98,7,445.1984,2,17.74,55300.0,888.3824,-0.2
4,HLDVDDSGK,98,9,493.2330,2,22.69,232000.0,984.4512,0.3


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine, lysine, proline, arginine, tyrosine: 15.9949 MKPRY
    variable deamidation of asparagine, glumatine: 0.984016 NQ
    variable methylation of lysine and arginine: 14.015650 KR
    variable pyro-glu from glutamine: -17.03 Q
    variable acetylation of lysine: 42.01 K


We'll then write this manipulated dataframe to a new file.

In [4]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks324['A'] = peaks324['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks324['C'] = peaks324['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks324['D'] = peaks324['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks324['E'] = peaks324['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks324['F'] = peaks324['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks324['G'] = peaks324['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks324['H'] = peaks324['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks324 output, there will be no isoleucines (they're lumped in with leucines)
peaks324['I'] = peaks324['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks324['K'] = peaks324['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks324['L'] = peaks324['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks324['M'] = peaks324['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks324['N'] = peaks324['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks324['P'] = peaks324['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks324['Q'] = peaks324['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks324['R'] = peaks324['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks324['S'] = peaks324['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks324['T'] = peaks324['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks324['V'] = peaks324['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks324['W'] = peaks324['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks324['Y'] = peaks324['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks324['c-carb'] = peaks324['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks324['m-oxid'] = peaks324['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a count function to enumerate the # of oxidized K's in each peptide
peaks324['k-oxid'] = peaks324['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of oxidized P's in each peptide
peaks324['p-oxid'] = peaks324['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of oxidized R's in each peptide
peaks324['r-oxid'] = peaks324['Peptide'].apply(lambda x: x.count('R(+15.99)'))

# use a count function to enumerate the # of oxidized Y's in each peptide
peaks324['y-oxid'] = peaks324['Peptide'].apply(lambda x: x.count('Y(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks324['n-deam'] = peaks324['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
#peaks324['q-deam'] = peaks324['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of methylated K's in each peptide
peaks324['k-meth'] = peaks324['Peptide'].apply(lambda x: x.count('K(+14.02)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks324['r-meth'] = peaks324['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# use a count function to enumerate the # of pyro glu Q's in each peptide
peaks324['q-pyro'] = peaks324['Peptide'].apply(lambda x: x.count('Q(-17.03)'))

# use a count function to enumerate the # of acetylation of K's in each peptide
peaks324['k-acet'] = peaks324['Peptide'].apply(lambda x: x.count('K(+42.01)'))

# create a column with 'stripped' peptide sequences using strip
peaks324['stripped_peptide'] = peaks324['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks324['stripped_length'] = peaks324['stripped_peptide'].apply(len)

# total the number of modifications in sequence
peaks324['ptm-total'] = peaks324['c-carb'] + peaks324['m-oxid'] + peaks324['k-oxid'] + peaks324['p-oxid'] \
+ peaks324['r-oxid'] + peaks324['y-oxid'] + peaks324['n-deam'] + peaks324['k-meth'] + peaks324['r-meth'] \
+ peaks324['q-pyro'] + peaks324['k-acet']

# calculate NAAF numerator for each peptide k
peaks324['NAAF_num.'] = peaks324['Area'] / peaks324['stripped_length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks324.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDN/TW_324_T5_trypsin_combine_PTMopt_DN80.csv")

# check out the results
peaks324.head()

,Peptide,ALC (%),length,m/z,z,RT,Area,Mass,ppm,A,...,y-oxid,n-deam,k-meth,r-meth,q-pyro,k-acet,stripped_peptide,stripped_length,ptm-total,NAAF_num.
0,KFDEETK,99,7,448.7220,2,17.90,1710000.0,895.4286,0.9,0,...,0,0,0,0,0,0,KFDEETK,7,0,244285.714286
1,TPEEEER,99,7,445.1992,2,16.15,52200.0,888.3824,1.6,0,...,0,0,0,0,0,0,TPEEEER,7,0,7457.142857
2,HLDVDDSGK,98,9,493.2327,2,25.39,180000.0,984.4512,-0.4,0,...,0,0,0,0,0,0,HLDVDDSGK,9,0,20000.000000
3,TPEEEER,98,7,445.1984,2,17.74,55300.0,888.3824,-0.2,0,...,0,0,0,0,0,0,TPEEEER,7,0,7900.000000
4,HLDVDDSGK,98,9,493.2330,2,22.69,232000.0,984.4512,0.3,0,...,0,0,0,0,0,0,HLDVDDSGK,9,0,25777.777778


In [5]:
# keep only stripped peptide I/L and NAAF
dn_324 = peaks324[['stripped_peptide', 'Area', 'NAAF_num.']]

dn_324.set_index('stripped_peptide')

# write modified dataframe to new txt file
dn_324.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/I-L_NAAFs/T5_324_trypsin_DN_peptides.csv")

dn_324.head()

,stripped_peptide,Area,NAAF_num.
0,KFDEETK,1710000.0,244285.714286
1,TPEEEER,52200.0,7457.142857
2,HLDVDDSGK,180000.0,20000.000000
3,TPEEEER,55300.0,7900.000000
4,HLDVDDSGK,232000.0,25777.777778


### Export txt file of entire (with modification terms) peptides only

In [6]:
# keep only peptide list with mods
dn_pep_324 = peaks324[['Peptide']]

# deduplice the lists
dn_modpep_324 = dn_pep_324.drop_duplicates()

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
dn_modpep_324.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PTM-cellular-compartment/to-combine/T5/TW_324_T5_trypsin_combine_PTMopt_DN_mod_peptides.txt", header=False, index=False)

# look at the stripped peptides
dn_modpep_324.head()

,Peptide
0,KFDEETK
1,TPEEEER
2,HLDVDDSGK
5,SPN(+.98)N(+.98)ETEER
6,VVEEVVR


In [7]:
# made a new dataframe that contains the sums of certain columns 
# in the stripped peptide dataframe above (for >80% ALC)

index = ['sample total']

data = {'A': peaks324['A'].sum(),
        'C': peaks324['C'].sum(),
        'D': peaks324['D'].sum(),
        'E': peaks324['E'].sum(),
        'F': peaks324['F'].sum(),
        'G': peaks324['G'].sum(),
        'H': peaks324['H'].sum(),
        'I': peaks324['I'].sum(),
        'K': peaks324['K'].sum(),
        'L': peaks324['L'].sum(),
        'M': peaks324['M'].sum(),
        'N': peaks324['N'].sum(),
        'P': peaks324['P'].sum(),
        'Q': peaks324['Q'].sum(),
        'R': peaks324['R'].sum(),
        'S': peaks324['S'].sum(),
        'T': peaks324['T'].sum(),
        'V': peaks324['V'].sum(),
        'W': peaks324['W'].sum(),
        'Y': peaks324['Y'].sum(),
        'c-carb': peaks324['c-carb'].sum(),
        'm-oxid': peaks324['m-oxid'].sum(),
        'k-oxid': peaks324['k-oxid'].sum(),
        'p-oxid': peaks324['p-oxid'].sum(),
        'r-oxid': peaks324['r-oxid'].sum(),
        'y-oxid': peaks324['y-oxid'].sum(),
        'n-deam': peaks324['n-deam'].sum(),
        'k-meth': peaks324['k-meth'].sum(),
        'r-meth': peaks324['r-meth'].sum(),
        'q-pyro': peaks324['q-pyro'].sum(),
        'k-acet': peaks324['k-acet'].sum(),
        'Total area': peaks324['Area'].sum(),
        'Total length': peaks324['stripped_length'].sum()
       }

totalpeaks324 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', \
                                            'y-oxid', 'n-deam', 'k-meth', 'r-meth', 'q-pyro', \
                                            'k-acet', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks324['% C w/ carb'] = totalpeaks324['c-carb'] / totalpeaks324['C'] 

# calculate percentage of M's that are oxidized
totalpeaks324['% M w/ oxid'] = totalpeaks324['m-oxid'] / totalpeaks324['M'] 

# calculate percentage of K's that are oxidized
totalpeaks324['% K w/ oxid'] = totalpeaks324['k-oxid'] / totalpeaks324['K'] 

# calculate percentage of P's that are oxidized
totalpeaks324['% P w/ oxid'] = totalpeaks324['p-oxid'] / totalpeaks324['P'] 

# calculate percentage of R's that are oxidized
totalpeaks324['% R w/ oxid'] = totalpeaks324['p-oxid'] / totalpeaks324['R'] 

# calculate percentage of Y's that are oxidized
totalpeaks324['% Y w/ oxid'] = totalpeaks324['y-oxid'] / totalpeaks324['Y'] 

# calculate percentage of N's that are deamidated
totalpeaks324['% N w/ deam'] = totalpeaks324['n-deam'] / totalpeaks324['N'] 

# calculate percentage of K's that are methylated
totalpeaks324['% K w/ meth'] = totalpeaks324['k-meth'] / totalpeaks324['K'] 

# calculate percentage of R's that are methylated
totalpeaks324['% R w/ meth'] = totalpeaks324['r-meth'] / totalpeaks324['R'] 

# calculate percentage of Q's that are pyro glu'd
totalpeaks324['% Q w/ pyro'] = totalpeaks324['q-pyro'] / totalpeaks324['Q'] 

# calculate percentage of K's that are acetylation
totalpeaks324['% K w/ acet'] = totalpeaks324['k-acet'] / totalpeaks324['K'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks324['NAAF denom.'] = totalpeaks324['Total area'] / totalpeaks324['Total length']

# write modified dataframe to new txt file
totalpeaks324.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDN/TW_324_T5_trypsin_combine_PTMopt_DN80_totals.csv")

totalpeaks324.head()

,A,C,D,E,F,G,H,I,K,L,...,% K w/ oxid,% P w/ oxid,% R w/ oxid,% Y w/ oxid,% N w/ deam,% K w/ meth,% R w/ meth,% Q w/ pyro,% K w/ acet,NAAF denom.
sample total,2424,286,1892,2611,1213,2036,268,0,2843,3611,...,0.071403,0.26543,0.235673,0.129666,0.33319,0.064369,0.051934,0.029263,0.142455,219544.700462


In [ ]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF80 = 219544.700462

# use NAAF >80% ALC to get NAAF factor
peaks324['NAAF factor'] = (peaks324['NAAF num.'])/NAAF80

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksAAPTM_324 = peaks324[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'k-oxid', 'p-oxid', 'r-oxid', 'y-oxid', 'n-deam', 'k-meth', 'r-meth', \
                                'q-pyro', 'k-acet']].copy()

# multiply the NAAF80 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_324['A-NAAF80'] = peaksAAPTM_324['A'] * peaks324['NAAF factor']
peaksAAPTM_324['C-NAAF80'] = peaksAAPTM_324['C'] * peaks324['NAAF factor']
peaksAAPTM_324['D-NAAF80'] = peaksAAPTM_324['D'] * peaks324['NAAF factor']
peaksAAPTM_324['E-NAAF80'] = peaksAAPTM_324['E'] * peaks324['NAAF factor']
peaksAAPTM_324['F-NAAF80'] = peaksAAPTM_324['F'] * peaks324['NAAF factor']
peaksAAPTM_324['G-NAAF80'] = peaksAAPTM_324['G'] * peaks324['NAAF factor']
peaksAAPTM_324['H-NAAF80'] = peaksAAPTM_324['H'] * peaks324['NAAF factor']
peaksAAPTM_324['I-NAAF80'] = peaksAAPTM_324['I'] * peaks324['NAAF factor']
peaksAAPTM_324['K-NAAF80'] = peaksAAPTM_324['K'] * peaks324['NAAF factor']
peaksAAPTM_324['L-NAAF80'] = peaksAAPTM_324['L'] * peaks324['NAAF factor']
peaksAAPTM_324['M-NAAF80'] = peaksAAPTM_324['M'] * peaks324['NAAF factor']
peaksAAPTM_324['N-NAAF80'] = peaksAAPTM_324['N'] * peaks324['NAAF factor']
peaksAAPTM_324['P-NAAF80'] = peaksAAPTM_324['P'] * peaks324['NAAF factor']
peaksAAPTM_324['Q-NAAF80'] = peaksAAPTM_324['Q'] * peaks324['NAAF factor']
peaksAAPTM_324['R-NAAF80'] = peaksAAPTM_324['R'] * peaks324['NAAF factor']
peaksAAPTM_324['S-NAAF80'] = peaksAAPTM_324['S'] * peaks324['NAAF factor']
peaksAAPTM_324['T-NAAF80'] = peaksAAPTM_324['T'] * peaks324['NAAF factor']
peaksAAPTM_324['V-NAAF80'] = peaksAAPTM_324['V'] * peaks324['NAAF factor']
peaksAAPTM_324['W-NAAF80'] = peaksAAPTM_324['W'] * peaks324['NAAF factor']
peaksAAPTM_324['Y-NAAF80'] = peaksAAPTM_324['Y'] * peaks324['NAAF factor']

# multiply the NAAF80 factor by the PTMs normalize its abundance by peak area and peptide length

peaksAAPTM_324['ccarb-NAAF80'] = peaksAAPTM_324['c-carb'] * peaksAAPTM_324['NAAF factor']
peaksAAPTM_324['moxid-NAAF80'] = peaksAAPTM_324['m-oxid'] * peaksAAPTM_324['NAAF factor']
peaksAAPTM_324['koxid-NAAF80'] = peaksAAPTM_324['k-oxid'] * peaksAAPTM_324['NAAF factor']
peaksAAPTM_324['poxid-NAAF80'] = peaksAAPTM_324['p-oxid'] * peaksAAPTM_324['NAAF factor']
peaksAAPTM_324['roxid-NAAF80'] = peaksAAPTM_324['r-oxid'] * peaksAAPTM_324['NAAF factor']
peaksAAPTM_324['yoxid-NAAF80'] = peaksAAPTM_324['y-oxid'] * peaksAAPTM_324['NAAF factor']
peaksAAPTM_324['ndeam-NAAF80'] = peaksAAPTM_324['n-deam'] * peaksAAPTM_324['NAAF factor']
peaksAAPTM_324['kmeth-NAAF80'] = peaksAAPTM_324['k-meth'] * peaksAAPTM_324['NAAF factor']
peaksAAPTM_324['rmeth-NAAF80'] = peaksAAPTM_324['r-meth'] * peaksAAPTM_324['NAAF factor']
peaksAAPTM_324['qpyro-NAAF80'] = peaksAAPTM_324['q-pyro'] * peaksAAPTM_324['NAAF factor']
peaksAAPTM_324['kacet-NAAF80'] = peaksAAPTM_324['k-acet'] * peaksAAPTM_324['NAAF factor']

# write the dataframe to a new csv
peaksAAPTM_324.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDN/TW_324_T5_trypsin_combine_PTMopt_DN80_NAAF.csv")

peaksAAPTM_324.head()

In [ ]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMs

index = ['sample total']

data = {'NAAF': peaksAAPTM_324['NAAF factor'].sum(),
        'A': peaksAAPTM_324['A-NAAF80'].sum(),
        'C': peaksAAPTM_324['C-NAAF80'].sum(),
        'D': peaksAAPTM_324['D-NAAF80'].sum(),
        'E': peaksAAPTM_324['E-NAAF80'].sum(),
        'F': peaksAAPTM_324['F-NAAF80'].sum(),
        'G': peaksAAPTM_324['G-NAAF80'].sum(),
        'H': peaksAAPTM_324['H-NAAF80'].sum(),
        'I': peaksAAPTM_324['I-NAAF80'].sum(),
        'K': peaksAAPTM_324['K-NAAF80'].sum(),
        'L': peaksAAPTM_324['L-NAAF80'].sum(),
        'M': peaksAAPTM_324['M-NAAF80'].sum(),
        'N': peaksAAPTM_324['N-NAAF80'].sum(),
        'P': peaksAAPTM_324['P-NAAF80'].sum(),
        'Q': peaksAAPTM_324['Q-NAAF80'].sum(),
        'R': peaksAAPTM_324['R-NAAF80'].sum(),
        'S': peaksAAPTM_324['S-NAAF80'].sum(),
        'T': peaksAAPTM_324['T-NAAF80'].sum(),
        'V': peaksAAPTM_324['V-NAAF80'].sum(),
        'W': peaksAAPTM_324['W-NAAF80'].sum(),
        'Y': peaksAAPTM_324['Y-NAAF80'].sum(),
        'c-carb': peaksAAPTM_324['ccarb-NAAF80'].sum(),
        'm-oxid': peaksAAPTM_324['moxid-NAAF80'].sum(),
        'k-oxid': peaksAAPTM_324['koxid-NAAF80'].sum(),
        'p-oxid': peaksAAPTM_324['poxid-NAAF80'].sum(),
        'r-oxid': peaksAAPTM_324['roxid-NAAF80'].sum(),
        'y-oxid': peaksAAPTM_324['yoxid-NAAF80'].sum(),
        'n-deam': peaksAAPTM_324['ndeam-NAAF80'].sum(),
        'k-meth': peaksAAPTM_324['kmeth-NAAF80'].sum(),
        'r-meth': peaksAAPTM_324['rmeth-NAAF80'].sum(),
        'q-pyro': peaksAAPTM_324['qpyro-NAAF80'].sum(),
        'k-acet': peaksAAPTM_324['kacet-NAAF80'].sum()
       }

totalpeaks80_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'k-oxid', 'p-oxid', \
                                           'r-oxid', 'y-oxid', 'n-deam', 'k-meth', 'r-meth', \
                                            'q-pyro', 'k-acet'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks80_NAAF['% C w/ carb.'] = totalpeaks80_NAAF['c-carb'] / totalpeaks80_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks80_NAAF['% M w/ oxid'] = totalpeaks80_NAAF['m-oxid'] / totalpeaks80_NAAF['M'] 

# calculate NAAF-corrected percentage of K's that are oxidized
totalpeaks80_NAAF['% K w/ oxid'] = totalpeaks80_NAAF['k-oxid'] / totalpeaks80_NAAF['K'] 

# calculate NAAF-corrected percentage of P's that are oxidized
totalpeaks80_NAAF['% P w/ oxid'] = totalpeaks80_NAAF['p-oxid'] / totalpeaks80_NAAF['P'] 

# calculate NAAF-corrected percentage of R's that are oxidized
totalpeaks80_NAAF['% R w/ oxid'] = totalpeaks80_NAAF['r-oxid'] / totalpeaks80_NAAF['R'] 

# calculate NAAF-corrected percentage of Y's that are oxidized
totalpeaks80_NAAF['% Y w/ oxid'] = totalpeaks80_NAAF['y-oxid'] / totalpeaks80_NAAF['Y'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks80_NAAF['% N w/ deam'] = totalpeaks80_NAAF['n-deam'] / totalpeaks80_NAAF['N'] 

# calculate NAAF-corrected percentage of K's that are methylated
totalpeaks80_NAAF['% K w/ meth'] = totalpeaks80_NAAF['k-meth'] / totalpeaks80_NAAF['K'] 

# calculate NAAF-corrected percentage of R's that are methylated
totalpeaks80_NAAF['% R w/ meth'] = totalpeaks80_NAAF['r-meth'] / totalpeaks80_NAAF['R'] 

# calculate NAAF-corrected percentage of Q's that are pyro glu'd
totalpeaks80_NAAF['% Q w/ pyro'] = totalpeaks80_NAAF['q-pyro'] / totalpeaks80_NAAF['Q'] 

# calculate NAAF-corrected percentage of K's that are methylated
totalpeaks80_NAAF['% K w/ acet'] = totalpeaks80_NAAF['k-acet'] / totalpeaks80_NAAF['K'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks80_NAAF['NAAF check'] = totalpeaks80_NAAF['NAAF'] / 219544.700462

# write modified dataframe to new txt file, same name + totals
totalpeaks80_NAAF.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDN/TW_324_T5_trypsin_combine_PTMopt_DN80_NAAF_totals.csv")

totalpeaks80_NAAF.head()

## Export stripped peptides >80% ALC

In [ ]:
# keep only stripped peptide column 
pep80 = peaks324[["stripped peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep80.to_csv("/home/millieginty/Documents/git-repos/TW_324_T5_trypsin_combine_PTMopt_DN80_stripped_peptides.txt", header=False, index=False)

# removing redundancy
peaks80dedup = pd.DataFrame.drop_duplicates(pep80)

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

peaks80dedup.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDN/TW_324_T5_trypsin_combine_PTMopt_DN80_nonredundant_stripped_peptides.txt", header=False, index=False)

print("# redundant stripped Peaks peptides >80% ALC", len(pep80))
print("# nonredundant stripped Peaks peptides >80% ALC", len(peaks80dedup))
print("average peptide length Peaks peptides >80% ALC", peaks324['stripped length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >80% ALC
pep80m = peaks324[["Peptide"]]

# deduplicate
pep80mdedup = pd.DataFrame.drop_duplicates(pep80m)

print("# redundant Peaks peptides >80% ALC", len(pep80m))
print("# nonredundant Peaks peptides", len(pep80mdedup))

# check
pep80.head()